## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns="EIN")
application_df=application_df.drop(columns="NAME")
#Deleted this as well.
application_df=application_df.drop(columns="SPECIAL_CONSIDERATIONS")

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:

#  YOUR CODE GOES HERE

In [5]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df.value_counts(["APPLICATION_TYPE"], sort=True))


APPLICATION_TYPE
T3                  27037
T4                   1542
T6                   1216
T5                   1173
T19                  1065
T8                    737
T7                    725
T10                   528
T9                    156
T13                    66
T12                    27
T2                     16
T14                     3
T25                     3
T29                     2
T15                     2
T17                     1
dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

import numpy as np
application_types_to_replace=application_df.APPLICATION_TYPE.value_counts()
val=application_types_to_replace[:8].index
application_df['APPLICATION_TYPE'] = application_df.APPLICATION_TYPE.where(application_df.APPLICATION_TYPE.isin(val), 'Other')
print(application_df.value_counts(["APPLICATION_TYPE"], sort=True))

APPLICATION_TYPE
T3                  27037
T4                   1542
T6                   1216
T5                   1173
T19                  1065
T8                    737
T7                    725
T10                   528
Other                 276
dtype: int64


In [7]:
#Check the ASK AMOUNT feature
print(application_df.value_counts(["ASK_AMT"], sort=True))

ASK_AMT   
5000          25398
63981             3
10478             3
15583             3
6725              3
              ...  
53268             1
53251             1
53248             1
53241             1
8597806340        1
Length: 8747, dtype: int64


# Next Cell modifed the ASK_AMT using 3 approaches and compares the results from each
1. Only consider those with ASK AMNT of $5000. This basically removes all entries with value count of less than 10 which leaves on ones with ASK_AMT of 5000. This yield the best results
2. Bin the ASK_AMT into 100 bins
3. Bin the ASK AMT into 1000 bins.
4. Drop all ASK_AMT with value count less than 2

In [8]:

#Option 1
#application_df=application_df[application_df.groupby('ASK_AMT').ASK_AMT.transform('count')>10].copy() 
#Option 2
#application_df['ASK_AMT'].value_counts(bins = 100, sort=False)
#Option 3
#application_df['ASK_AMT'].value_counts(bins = 1000, sort=False)
#Option 4
application_df=application_df[application_df.groupby('ASK_AMT').ASK_AMT.transform('count')>1].copy() 
#Option 5 No change to ASK_AMT column


In [9]:
print(application_df.value_counts(["ASK_AMT"], sort=True))

ASK_AMT
5000       25398
10478          3
15583          3
63981          3
6725           3
           ...  
16071          2
16200          2
16435          2
16482          2
1427468        2
Length: 152, dtype: int64


In [10]:
# Look at CLASSIFICATION value counts for binning
application_df=application_df[application_df.groupby('CLASSIFICATION').CLASSIFICATION.transform('count')>1].copy() 

In [11]:
print(application_df.value_counts(["CLASSIFICATION"], sort=True))

CLASSIFICATION
C1000             12381
C2000              4617
C1200              3939
C2100              1595
C3000              1520
C7000               524
C1700               221
C4000               122
C5000               106
C1270                89
C2700                75
C2800                62
C7100                61
C1280                47
C1300                40
C1230                35
C2300                28
C1240                27
C1400                25
C7200                20
C6000                14
C1250                13
C8000                13
C7120                12
C8200                10
C1278                10
C1238                 9
C1235                 9
C1237                 9
C1500                 6
C1720                 6
C2400                 5
C1257                 5
C7210                 5
C1600                 4
C4100                 4
C1800                 3
C1260                 3
C1246                 2
C1256                 2
C1267                 2
C

In [12]:
#Drop all values with value count <=1
application_df = application_df[application_df.groupby('CLASSIFICATION').CLASSIFICATION.transform('count')>1].copy() 
application_df['CLASSIFICATION'].value_counts()

C1000    12381
C2000     4617
C1200     3939
C2100     1595
C3000     1520
C7000      524
C1700      221
C4000      122
C5000      106
C1270       89
C2700       75
C2800       62
C7100       61
C1280       47
C1300       40
C1230       35
C2300       28
C1240       27
C1400       25
C7200       20
C6000       14
C8000       13
C1250       13
C7120       12
C8200       10
C1278       10
C1235        9
C1237        9
C1238        9
C1500        6
C1720        6
C7210        5
C1257        5
C2400        5
C1600        4
C4100        4
C1260        3
C1800        3
C1267        2
C1256        2
C1246        2
C0           2
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`

application_df=pd.get_dummies(application_df)
application_df.dtypes

STATUS                    int64
ASK_AMT                   int64
IS_SUCCESSFUL             int64
APPLICATION_TYPE_Other    uint8
APPLICATION_TYPE_T10      uint8
                          ...  
INCOME_AMT_10M-50M        uint8
INCOME_AMT_1M-5M          uint8
INCOME_AMT_25000-99999    uint8
INCOME_AMT_50M+           uint8
INCOME_AMT_5M-10M         uint8
Length: 78, dtype: object

In [14]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
input_dim=X_train_scaled[0].size

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer

nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=input_dim))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the modelnn_model = tf.keras.models.Sequential()
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                6240      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,701
Trainable params: 8,701
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
#Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
602/602 [==============================] - 3s 3ms/step - loss: 0.5568 - accuracy: 0.7310
Epoch 2/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5330 - accuracy: 0.7454
Epoch 3/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5308 - accuracy: 0.7482
Epoch 4/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5277 - accuracy: 0.7478
Epoch 5/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5258 - accuracy: 0.7498
Epoch 6/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5262 - accuracy: 0.7485
Epoch 7/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5249 - accuracy: 0.7509
Epoch 8/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5234 - accuracy: 0.7488
Epoch 9/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5231 - accuracy: 0.7511
Epoch 10/50
602/602 [==============================] - 2s 3ms/step - loss: 0.5230 - accuracy: 0.7501

In [19]:
# Evaluate the model using the test data
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

201/201 - 1s - loss: 0.5401 - accuracy: 0.7472 - 769ms/epoch - 4ms/step
Loss: 0.5400767922401428, Accuracy: 0.7472356557846069


ASK AMOUNT is only fixed at 5000 amount

199/199 - 1s - loss: 0.5345 - accuracy: 0.7502 - 719ms/epoch - 4ms/step
Loss: 0.5344988107681274, Accuracy: 0.7501576542854309

Without ASK amount limit to 5000, but limit value count to > 1

201/201 - 1s - loss: 0.5493 - accuracy: 0.7466 - 637ms/epoch - 3ms/step
Loss: 0.5492763519287109, Accuracy: 0.7466126680374146

With 1000 bins for ASK AMT:

268/268 - 1s - loss: 0.5664 - accuracy: 0.7238 - 875ms/epoch - 3ms/step
Loss: 0.5663652420043945, Accuracy: 0.7237717509269714

With 100 bins for ASK_AMT

268/268 - 1s - loss: 0.5755 - accuracy: 0.7226 - 810ms/epoch - 3ms/step
Loss: 0.575512707233429, Accuracy: 0.7226047515869141

Without any filter on ASK_AMT

201/201 - 1s - loss: 0.5401 - accuracy: 0.7472 - 769ms/epoch - 4ms/step
Loss: 0.5400767922401428, Accuracy: 0.7472356557846069


Answer sheet

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391

In [21]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn_model.save("nn_model.h5")